analysis the primary forest change inside and outside the protected area

In [2]:
import numpy as np
from osgeo import gdal, gdal_array, gdalconst, osr
import os
from os.path import join
import pandas as pd


if __name__ == '__main__':
    
    pwd = os.getcwd()
    rootpath = os.path.abspath(os.path.join(pwd, '..'))

    list_year = np.arange(1996, 2023)

    img_haiti_pa = gdal_array.LoadFile(join(rootpath, 'data', 'protected_area', 'haiti_pa_compress.tif'))
    img_dr_pa = gdal_array.LoadFile(join(rootpath, 'data', 'protected_area', 'dr_pa_compress.tif'))

    img_country = gdal_array.LoadFile(join(rootpath, 'data', 'hispaniola_polygon', 'countryid_hispaniola.tif'))

    df_pa = pd.DataFrame(columns=['year', 'haiti_pf_num', 'haiti_pf_inside_pa', 'haiti_pf_outside_pa',
                                  'haiti_pf_wet_num', 'haiti_pf_wet_inside_pa', 'haiti_pf_wet_outside_pa',
                                  'haiti_pf_dry_num', 'haiti_pf_dry_inside_pa', 'haiti_pf_dry_outside_pa',
                                  'dr_pf_num', 'dr_pf_inside_pa', 'dr_pf_outside_pa',
                                  'dr_pf_wet_num', 'dr_pf_wet_inside_pa', 'dr_pf_wet_outside_pa',
                                  'dr_pf_dry_num', 'dr_pf_dry_inside_pa', 'dr_pf_dry_outside_pa'
                                  ],
                         index=np.arange(0, len(list_year)))

    for i_year in range(0, len(list_year)):
        year = list_year[i_year]

        land_cover = gdal_array.LoadFile(join(rootpath, 'data', 'hispaniola_lc', f'hispaniola_lc_{year}.tif'))
        
        # Haiti
        count_pf_wet_haiti = np.count_nonzero(land_cover[(land_cover == 2) & (img_country == 1)])
        count_pf_wet_haiti_inside_pa = np.count_nonzero(land_cover[(land_cover == 2) & (img_country == 1) & (img_haiti_pa > 0)])
        count_pf_wet_haiti_outside_pa = np.count_nonzero(land_cover[(land_cover == 2) & (img_country == 1) & (img_haiti_pa == 0)])

        count_pf_dry_haiti = np.count_nonzero(land_cover[(land_cover == 3) & (img_country == 1)])
        count_pf_dry_haiti_inside_pa = np.count_nonzero(land_cover[(land_cover == 3) & (img_country == 1) & (img_haiti_pa > 0)])
        count_pf_dry_haiti_outside_pa = np.count_nonzero(land_cover[(land_cover == 3) & (img_country == 1) & (img_haiti_pa == 0)])

        count_pf_haiti = count_pf_wet_haiti + count_pf_dry_haiti
        count_pf_haiti_inside_pa = count_pf_wet_haiti_inside_pa + count_pf_dry_haiti_inside_pa
        count_pf_haiti_outside_pa = count_pf_wet_haiti_outside_pa + count_pf_dry_haiti_outside_pa
        
        # Dominican Republic
        count_pf_wet_dr = np.count_nonzero(land_cover[(land_cover == 2) & (img_country == 2)])
        count_pf_wet_dr_inside_pa = np.count_nonzero(land_cover[(land_cover == 2) & (img_country == 2) & (img_dr_pa > 0)])
        count_pf_wet_dr_outside_pa = np.count_nonzero(land_cover[(land_cover == 2) & (img_country == 2) & (img_dr_pa == 0)])

        count_pf_dry_dr = np.count_nonzero(land_cover[(land_cover == 3) & (img_country == 2)])
        count_pf_dry_dr_inside_pa = np.count_nonzero(land_cover[(land_cover == 3) & (img_country == 2) & (img_dr_pa > 0)])
        count_pf_dry_dr_outside_pa = np.count_nonzero(land_cover[(land_cover == 3) & (img_country == 2) & (img_dr_pa == 0)])

        count_pf_dr = count_pf_wet_dr + count_pf_dry_dr
        count_pf_dr_inside_pa = count_pf_wet_dr_inside_pa + count_pf_dry_dr_inside_pa
        count_pf_dr_outside_pa = count_pf_wet_dr_outside_pa + count_pf_dry_dr_outside_pa

        print(year, count_pf_haiti, count_pf_haiti_inside_pa, count_pf_haiti_outside_pa)
        print(year, count_pf_dr, count_pf_dr_inside_pa, count_pf_dr_outside_pa)

        df_pa.loc[i_year, 'year'] = year

        df_pa.loc[i_year, 'haiti_pf_num'] = count_pf_haiti
        df_pa.loc[i_year, 'haiti_pf_inside_pa'] = count_pf_haiti_inside_pa
        df_pa.loc[i_year, 'haiti_pf_outside_pa'] = count_pf_haiti_outside_pa

        df_pa.loc[i_year, 'haiti_pf_wet_num'] = count_pf_wet_haiti
        df_pa.loc[i_year, 'haiti_pf_wet_inside_pa'] = count_pf_wet_haiti_inside_pa
        df_pa.loc[i_year, 'haiti_pf_wet_outside_pa'] = count_pf_wet_haiti_outside_pa

        df_pa.loc[i_year, 'haiti_pf_dry_num'] = count_pf_dry_haiti
        df_pa.loc[i_year, 'haiti_pf_dry_inside_pa'] = count_pf_dry_haiti_inside_pa
        df_pa.loc[i_year, 'haiti_pf_dry_outside_pa'] = count_pf_dry_haiti_outside_pa

        df_pa.loc[i_year, 'dr_pf_num'] = count_pf_dr
        df_pa.loc[i_year, 'dr_pf_inside_pa'] = count_pf_dr_inside_pa
        df_pa.loc[i_year, 'dr_pf_outside_pa'] = count_pf_dr_outside_pa

        df_pa.loc[i_year, 'dr_pf_wet_num'] = count_pf_wet_dr
        df_pa.loc[i_year, 'dr_pf_wet_inside_pa'] = count_pf_wet_dr_inside_pa
        df_pa.loc[i_year, 'dr_pf_wet_outside_pa'] = count_pf_wet_dr_outside_pa

        df_pa.loc[i_year, 'dr_pf_dry_num'] = count_pf_dry_dr
        df_pa.loc[i_year, 'dr_pf_dry_inside_pa'] = count_pf_dry_dr_inside_pa
        df_pa.loc[i_year, 'dr_pf_dry_outside_pa'] = count_pf_dry_dr_outside_pa

1996 227193 164359 62834
1996 3813570 3300584 512986
1997 224483 161911 62572
1997 3764524 3254582 509942
1998 221858 159889 61969
1998 3711052 3205696 505356
1999 215831 155612 60219
1999 3636176 3142101 494075
2000 209129 149528 59601
2000 3604243 3117141 487102
2001 205199 145899 59300
2001 3569609 3084215 485394
2002 203720 144645 59075
2002 3553297 3069095 484202
2003 202985 144019 58966
2003 3545440 3062068 483372
2004 201401 142790 58611
2004 3530490 3050520 479970
2005 198326 140157 58169
2005 3448983 2973101 475882
2006 196507 138540 57967
2006 3388689 2915811 472878
2007 194321 136486 57835
2007 3363071 2892971 470100
2008 191737 134651 57086
2008 3346870 2881374 465496
2009 190395 133603 56792
2009 3337133 2876312 460821
2010 189603 132941 56662
2010 3326806 2868652 458154
2011 188887 132336 56551
2011 3317862 2861712 456150
2012 185930 129714 56216
2012 3309038 2855294 453744
2013 179824 124566 55258
2013 3260673 2810060 450613
2014 178232 123372 54860
2014 3209701 2762743 

In [3]:
    df_pa

,year,haiti_pf_num,haiti_pf_inside_pa,haiti_pf_outside_pa,haiti_pf_wet_num,haiti_pf_wet_inside_pa,haiti_pf_wet_outside_pa,haiti_pf_dry_num,haiti_pf_dry_inside_pa,haiti_pf_dry_outside_pa,dr_pf_num,dr_pf_inside_pa,dr_pf_outside_pa,dr_pf_wet_num,dr_pf_wet_inside_pa,dr_pf_wet_outside_pa,dr_pf_dry_num,dr_pf_dry_inside_pa,dr_pf_dry_outside_pa
0,1996,227193,164359,62834,219052,159949,59103,8141,4410,3731,3813570,3300584,512986,3205273,2778782,426491,608297,521802,86495
1,1997,224483,161911,62572,216416,157519,58897,8067,4392,3675,3764524,3254582,509942,3157191,2733424,423767,607333,521158,86175
2,1998,221858,159889,61969,213868,155515,58353,7990,4374,3616,3711052,3205696,505356,3105146,2685128,420018,605906,520568,85338
3,1999,215831,155612,60219,207945,151323,56622,7886,4289,3597,3636176,3142101,494075,3030910,2621764,409146,605266,520337,84929
4,2000,209129,149528,59601,201595,145468,56127,7534,4060,3474,3604243,3117141,487102,3006662,2601541,405121,597581,515600,81981
5,2001,205199,145899,59300,197822,141903,55919,7377,3996,3381,3569609,3084215,485394,2972807,2569220,403587,596802,514995,81807
6,2002,203720,144645,59075,196663,140899,55764,7057,3746,3311,3553297,3069095,484202,2957501,2554784,402717,595796,514311,81485
7,2003,202985,144019,58966,196125,140443,55682,6860,3576,3284,3545440,3062068,483372,2950460,2548384,402076,594980,513684,81296
8,2004,201401,142790,58611,194753,139391,55362,6648,3399,3249,3530490,3050520,479970,2940688,2540085,400603,589802,510435,79367
9,2005,198326,140157,58169,191873,136839,55034,6453,3318,3135,3448983,2973101,475882,2862333,2464121,398212,586650,508980,77670


In [4]:
    output_filename = join(rootpath, 'results', 'pa_pf_count.xlsx')
    df_pa.to_excel(output_filename)